In [8]:
# Simple Daikon-style invariant checker
# Complete the provided code around lines 28 and 44
# Do not modify the __repr__ functions.
# Modify only the classes Range and Invariants,
# if you need additional functions, make sure
# they are inside the classes.

import sys
import math
import random

def square_root(x, eps = 0.00001):
    assert x >= 0
    y = math.sqrt(x)
    assert abs(square(y) - x) <= eps
    return y
    
def square(x):
    return x * x

def double(x):
    return 2*abs(x)


class Range:
    """The Range class tracks the types and value ranges for a single variable."""
    
    def __init__(self):
        self.min  = None  # Minimum value seen
        self.max  = None  # Maximum value seen

    def track(self, value):
        """This method is invoked for every value."""
        if self.min is None and self.max is None:
            self.min = value
            self.max = value
        elif value < self.min:
            self.min = value
        elif value > self.max:
            self.max = value
        return
            
    def __repr__(self):
        return repr(self.min) + ".." + repr(self.max)


# The Invariants class tracks all Ranges for all variables seen.
class Invariants:
    
    def __init__(self):
        # Mapping (Function Name) -> (Event type) -> (Variable Name)
        # e.g. self.vars["sqrt"]["call"]["x"] = Range()
        # holds the range for the argument x when calling sqrt(x)
        self.vars = {}
        
    def track(self, frame, event, arg):
        # Track all variables and their values
        # If the event is "return", the return value
        # is kept in the 'arg' argument to this function.
        if event == "call" or event == "return":
            function_name = frame.f_code.co_name
            
            # This check is only necessary when running this code in Jupyter notebook
            if function_name.startswith("__") or function_name in ["run_code", "<module>",
                                                                  "pre_run_code_hook", "user_global_ns"]:
                return
            
            if function_name not in self.vars:
                self.vars[function_name] = {"call": {}, "return": {}}
            
            if event == "call":
                for variable in frame.f_locals:
                    try:
                        self.vars[function_name][event][variable].track(frame.f_locals[variable])
                    except KeyError:
                        self.vars[function_name][event][variable] = Range()
                        self.vars[function_name][event][variable].track(frame.f_locals[variable])
                        
            else:
                for variable in frame.f_locals:
                    if variable not in self.vars[function_name]["call"]:
                        try:
                            self.vars[function_name][event][variable].track(frame.f_locals[variable])
                        except KeyError:
                            self.vars[function_name][event][variable] = Range()
                            self.vars[function_name][event][variable].track(frame.f_locals[variable])
                ret = arg
                try:
                    self.vars[function_name][event]["ret"].track(ret)
                except KeyError:
                    self.vars[function_name][event]["ret"] = Range()
                    self.vars[function_name][event]["ret"].track(ret)

        return
    
    def __repr__(self):
        # Return the tracked invariants
        s = ""
        for function, events in self.vars.iteritems():
            for event, vars in events.iteritems():
                s += event + " " + function + ":\n"
                # continue
                
                for var, range in vars.iteritems():
                    s += "    assert "
                    if range.min == range.max:
                        s += var + " == " + repr(range.min)
                    else:
                        s += repr(range.min) + " <= " + var + " <= " + repr(range.max)
                    s += "\n"
                
        return s

invariants = Invariants()
    
def traceit(frame, event, arg):
    invariants.track(frame, event, arg)
    return traceit

sys.settrace(traceit)

# Tester. Increase the range for more precise results when running locally
eps = 0.000001
for i in range(1, 10):
    r = int(random.random() * 1000) # An integer value between 0 and 999.99
    z = square_root(r, eps)
    z = square(z)

for i in [3, 0, -10]:
    double(i)

sys.settrace(None)

print invariants

return double:
    assert 0 <= ret <= 20
call double:
    assert -10 <= x <= 3
return square_root:
    assert 0.0 <= y <= 27.694764848252458
    assert 0.0 <= ret <= 27.694764848252458
call square_root:
    assert 0 <= x <= 767
    assert eps == 1e-06
return square:
    assert 0.0 <= ret <= 767.0
call square:
    assert 0.0 <= x <= 27.694764848252458

